In [ ]:
import numpy as np

def get_init_prob_np(filename="state_weights.txt"):
    with open(filename, "r") as f:
        lines = f.readlines()

    states = []
    weights = []

    for line in lines[2:]:
        parts = line.strip().split()
        states.append(parts[0].strip('"'))
        weights.append(int(parts[1]))

    weights = np.array(weights, dtype=float)
    weights /= weights.sum()

    return dict(zip(states, weights))

def get_sensor_model(filename="state_observation_weights.txt", verbose=False):
    with open(filename, "r") as f:
        lines = f.readlines()

    _, _, _, default_obs_weight = map(int, lines[1].split())
    sensor_model = {}
    state_set = set()
    observation_set = set()

    for line in lines[2:]:
        parts = line.strip().split()
        state = parts[0].strip('"')
        observation = parts[1].strip('"')
        weight = int(parts[2])
        state_set.add(state)
        observation_set.add(observation)

        # Update the state probabilities dictionary
        if state not in sensor_model:
            sensor_model[state] = {}
        sensor_model[state][observation] = sensor_model[state].get(observation, 0) + weight

    # Add missing states with all observations using the default weight
    missing_states = state_set.symmetric_difference(sensor_model.keys())
    for state in missing_states:
        sensor_model[state] = {observation: default_obs_weight for observation in observation_set}

    # Add missing observations with default weight for all states
    for state in state_set:
        missing_observations = observation_set.symmetric_difference(sensor_model[state].keys())
        for observation in missing_observations:
            sensor_model[state][observation] = default_obs_weight

    # Calculate the probabilities for each state-observation pair
    for state, obs_dict in sensor_model.items():
        total_weight = sum(obs_dict.values())
        for obs in obs_dict:
            sensor_model[state][obs] = obs_dict[obs] / total_weight

    if verbose:
        for state, obs_dict in sensor_model.items():
            print('-------------------------')
            for obs, prob in obs_dict.items():
                print(f'State = {state}, Obs = {obs}, Prob = {round(prob, 2)}')

    return sensor_model

def get_transition_model(filename="state_action_state_weights.txt", verbose=False):
    with open(filename, "r") as f:
        lines = f.readlines()

    n_lines, n_states, n_actions, default_transition_weight = map(int, lines[1].split())

    transition_model = {}
    state_set = set()
    action_set = set()

    for line in lines[2:]:
        parts = line.strip().split()
        state = parts[0].strip('"')
        action = parts[1].strip('"')
        new_state = parts[2].strip('"')
        weight = int(parts[3])

        state_set.add(state)
        action_set.add(action)

        if state not in transition_model:
            transition_model[state] = {}
        if action not in transition_model[state]:
            transition_model[state][action] = {}

        transition_model[state][action][new_state] = weight

    # Add missing states, actions, and new states with default probabilities
    for state in state_set:
        for action in action_set:
            if action not in transition_model[state]:
                transition_model[state][action] = {}
            actions = transition_model[state][action]
            all_possible_states = set(state_set)
            for new_state in all_possible_states:
                if new_state not in actions:
                    transition_model[state][action][new_state] = default_transition_weight

    # Calculate the probabilities for each state-action-state pair
    for state, action_dict in transition_model.items():
        for action, new_state_dict in action_dict.items():
            total_weight = sum(new_state_dict.values())
            for new_state, weight in new_state_dict.items():
                transition_model[state][action][new_state] = weight / total_weight

    if verbose:
        for state, action_dict in transition_model.items():
            print('-------------------------------')
            print(f'State = {state}')
            for action, new_state_dict in action_dict.items():
                print(f'\tAction = {action}')
                for new_state, weight in new_state_dict.items():
                    print(f'\t\tState = {new_state}, Prob = {round(weight, 2)}')

    return transition_model

def get_observations(filename="observation_actions.txt"):
    with open(filename, "r") as f:
        lines = f.readlines()

    observations = []

    for i in range(2, len(lines)):
        line = lines[i]
        parts = line.strip().split()
        if len(parts) == 2:
            state = parts[0].strip('"')
            action = parts[1].strip('"')
            observations.append((state, action))
        else:
            state = parts[0].strip('"')
            observations.append((state,))

    return observations

def viterbi_np(init_prob, sensor_model, transition_model, observations, verbose=False):
    num_observations = len(observations)
    states = list(init_prob.keys())
    num_states = len(states)
    state_index = {state: i for i, state in enumerate(states)}

    alpha = np.zeros((num_observations, num_states))
    backpointer = np.zeros((num_observations, num_states), dtype=int)

    # Initial probabilities
    for i, state in enumerate(states):
        alpha[0, i] = init_prob[state] * sensor_model[state][observations[0][0]]

    # Main Viterbi loop
    for t in range(1, num_observations):
        for i, state in enumerate(states):
            max_prob = -1
            best_state = 0
            for j, prev_state in enumerate(states):
                prob = alpha[t - 1, j] * transition_model[prev_state][observations[t - 1][1]][state] * sensor_model[state][observations[t][0]]
                if prob > max_prob:
                    max_prob = prob
                    best_state = j
            alpha[t, i] = max_prob
            backpointer[t, i] = best_state

    # Backtrack
    most_likely_sequence = []
    last_state = np.argmax(alpha[-1, :])
    most_likely_sequence.append(states[last_state])

    for t in range(num_observations - 1, 0, -1):
        last_state = backpointer[t, last_state]
        most_likely_sequence.insert(0, states[last_state])

    return most_likely_sequence

# Main script
init_prob = get_init_prob_np("state_weights.txt")
sensor_model = get_sensor_model("state_observation_weights.txt", verbose=False)
transition_model = get_transition_model("state_action_state_weights.txt", verbose=False)
observations = get_observations("observation_actions.txt")

result = viterbi_np(init_prob, sensor_model, transition_model, observations, verbose=False)

with open('states.txt', 'w') as f:
    f.write('states\n')
    f.write(str(len(result))+"\n")
    for i in result:
        f.write(f"\"{i}\"\n")
